In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Load product information data
product_info_path = '/content/drive/MyDrive/INFOSYS723/Project/product_info.csv'
product_info_df = pd.read_csv(product_info_path)

In [3]:
review_files = [
    '/content/drive/MyDrive/INFOSYS723/Project/reviews_0-250.csv',
    '/content/drive/MyDrive/INFOSYS723/Project/reviews_250-500.csv',
    '/content/drive/MyDrive/INFOSYS723/Project/reviews_500-750.csv',
    '/content/drive/MyDrive/INFOSYS723/Project/reviews_750-1250.csv',
    '/content/drive/MyDrive/INFOSYS723/Project/reviews_1250-end.csv'
]

In [4]:
reviews = pd.concat([pd.read_csv(f) for f in review_files], ignore_index=True)

<ipython-input-4-07092264ab93>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.concat([pd.read_csv(f) for f in review_files], ignore_index=True)
<ipython-input-4-07092264ab93>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.concat([pd.read_csv(f) for f in review_files], ignore_index=True)
<ipython-input-4-07092264ab93>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.concat([pd.read_csv(f) for f in review_files], ignore_index=True)


In [5]:
reviews.drop_duplicates(inplace=True)

In [6]:
reviews.dropna(inplace=True)

In [7]:
reviews['submission_time'] = pd.to_datetime(reviews['submission_time'])

In [8]:
reviews['skin_type'] = reviews['skin_type'].replace({'dryy': 'dry', 'oilyy': 'oily'})

In [9]:
reviews.drop_duplicates(inplace=True)

In [10]:
reviews.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
Q1 = reviews['price_usd'].quantile(0.25)
Q3 = reviews['price_usd'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

reviews = reviews[(reviews['price_usd'] >= lower_bound) & (reviews['price_usd'] <= upper_bound)]

In [12]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283343 entries, 5 to 1094377
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   author_id                 283343 non-null  object        
 1   rating                    283343 non-null  int64         
 2   is_recommended            283343 non-null  float64       
 3   helpfulness               283343 non-null  float64       
 4   total_feedback_count      283343 non-null  int64         
 5   total_neg_feedback_count  283343 non-null  int64         
 6   total_pos_feedback_count  283343 non-null  int64         
 7   submission_time           283343 non-null  datetime64[ns]
 8   review_text               283343 non-null  object        
 9   review_title              283343 non-null  object        
 10  skin_tone                 283343 non-null  object        
 11  eye_color                 283343 non-null  object        
 12  s

In [13]:
reviews['review_text']

5          The scent isn’t my favourite but it works grea...
6          I’ll give this 2 stars for nice packaging and ...
8          I love this stuff. I first had the sample size...
9          I purchased the Sweet Candy scent at my local ...
21         Just like  Aquaphor just get something cheaper...
                                 ...                        
1094366    I’ve been taking SuperPower for the past month...
1094367    I love that you have to mix the two items for ...
1094368    I bought this set from drunk elephants website...
1094376    Really good for dry patchy skin!!! My skin has...
1094377    Have used this lotion for two weeks now, skin ...
Name: review_text, Length: 283343, dtype: object

In [14]:
import nltk
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Lowercasing, Removing punctuation, Tokenization, Removing stopwords,Lemmatization, Removing numbers, Handling contractions, Removing extra whitespaces

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if not word.isdigit()]

    # Handling contractions
    contractions = {
        "isn't": "is not",
        "aren't": "are not",
        "can't": "cannot",
        "couldn't": "could not",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'll": "he will",
        "he's": "he is",
        "i'd": "I would",
        "i'll": "I will",
        "i'm": "I am",
        "i've": "I have",
        "isn't": "is not",
        "it's": "it is",
        "let's": "let us",
        "mustn't": "must not",
        "shan't": "shall not",
        "she'd": "she would",
        "she'll": "she will",
        "she's": "she is",
        "shouldn't": "should not",
        "that's": "that is",
        "there's": "there is",
        "they'd": "they would",
        "they'll": "they will",
        "they're": "they are",
        "they've": "they have",
        "we'd": "we would",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "where's": "where is",
        "who'd": "who would",
        "who'll": "who will",
        "who're": "who are",
        "who's": "who is",
        "who've": "who have",
        "won't": "will not",
        "wouldn't": "would not",
        "you'd": "you would",
        "you'll": "you will",
        "you're": "you are",
        "you've": "you have"
    }
    tokens = [contractions[word] if word in contractions else word for word in tokens]

    # Removing extra whitespaces
    tokens = [word.strip() for word in tokens if word.strip()]

    # Joining tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


    # Removing HTML tags and URLs: NOT REQUIRED EXAMPLE
    #tokens = [re.sub(r'<[^>]+>', '', word) for word in tokens]
    #tokens = [re.sub(r'http\S+', '', word) for word in tokens]

In [16]:
reviews['sentiment'] = reviews['rating'].apply(lambda x: 'positive' if x in [4, 5] else 'negative')

In [17]:
reviews.head()

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,sentiment
5,42802569154,4,1.0,1.000000,1,0,1,2023-03-19,The scent isn’t my favourite but it works grea...,Great!,lightMedium,brown,normal,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,positive
6,6941883808,2,0.0,0.250000,8,6,2,2023-03-19,I’ll give this 2 stars for nice packaging and ...,Dried my lips out and clogged my pores,light,blue,combination,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,negative
8,7656791726,5,1.0,1.000000,1,0,1,2023-03-18,I love this stuff. I first had the sample size...,Must have.,light,blue,normal,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,positive
9,38727834382,5,1.0,1.000000,2,0,2,2023-03-18,I purchased the Sweet Candy scent at my local ...,RUN TO PURCHASE,light,brown,normal,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,positive
21,46007186639,1,0.0,0.727273,11,3,8,2023-03-14,Just like Aquaphor just get something cheaper...,Overrated!,fair,blue,normal,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,negative


In [ ]:
print(reviews.columns)


Index(['author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd', 'sentiment'],
      dtype='object')


In [18]:
# Assuming 'reviews' is your DataFrame containing the cleaned reviews data

# Specify the file path for the CSV file
finalcleaned_reviews_path = '/content/drive/MyDrive/INFOSYS723/Project/finalcleaned_reviews.csv'

# Save the DataFrame to a CSV file
reviews.to_csv(finalcleaned_reviews_path, index=False)

print("Final cleaned reviews data saved to:", finalcleaned_reviews_path)


Final cleaned reviews data saved to: /content/drive/MyDrive/INFOSYS723/Project/finalcleaned_reviews.csv


In [30]:
import spacy

# Load pre-trained NER model
nlp = spacy.load("en_core_web_sm")

# Path to the file containing cleaned reviews
reviews_path = '/content/drive/MyDrive/INFOSYS723/Project/finalcleaned_reviews.csv'

# Function to extract named entities from text
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Process first 1000 reviews and extract named entities
with open(reviews_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i >= 1000:
            break
        review_text = line.strip()  # Assuming each line contains a review text
        entities = extract_entities(review_text)
        print("Review Text:", review_text)
        print("Named Entities:", entities)
        print()


Review Text: author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,sentiment
Named Entities: [('author_id', 'ORG'), ('total_pos_feedback_count', 'PERSON')]

Review Text: 42802569154,4,1.0,1.0,1,0,1,2023-03-19,"The scent isn’t my favourite but it works great! I put it on every night before I go to sleep and every morning I wake up with smooth, moisturizer and soft. Packaging is amazing as well",Great!,lightMedium,brown,normal,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitamin C,LANEIGE,24.0,positive
Named Entities: [('42802569154,4,1.0,1.0,1,0,1,2023-03-19,"The', 'CARDINAL'), ('every night', 'TIME'), ('every morning', 'TIME'), ('Vitamin C', 'ORG')]

Review Text: 6941883808,2,0.0,0.25,8,6,2,2023-03-19,"I’ll give this 2 stars for nice packaging and lovely scent. Upon initial application, it f

##Here are some questions you can solve using the pre-trained NER model

QUESTION 1:What are the most frequently mentioned entities in the reviews?



In [31]:
from collections import Counter

# Initialize a Counter to count entity occurrences
entity_counter = Counter()

# Process reviews file and extract named entities
with open(reviews_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i >= 1000:
            break
        review_text = line.strip()  # Assuming each line contains a review text
        doc = nlp(review_text)
        # Increment counter for each named entity in the document
        for ent in doc.ents:
            entity_counter[ent.text] += 1

# Print the most common entities
most_common_entities = entity_counter.most_common(10)  # Get top 10 most common entities
print("Top 10 Most Common Entities:")
for entity, count in most_common_entities:
    print(f"{entity}: {count}")


Top 10 Most Common Entities:
Vitamin C: 999
the day: 91
first: 63
night: 62
one: 43
Laneige: 39
Vaseline: 37
mediumTan: 35
Sephora: 29
the morning: 26


QUESTION 2:What are the different types of entities mentioned in the reviews?

In [32]:
# Initialize a set to store unique entity types
entity_types = set()

# Process reviews file and extract named entity types
with open(reviews_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i >= 1000:
            break
        review_text = line.strip()  # Assuming each line contains a review text
        doc = nlp(review_text)
        # Add each entity type to the set
        for ent in doc.ents:
            entity_types.add(ent.label_)

# Print the unique entity types
print("Unique Entity Types:")
print(entity_types)


Unique Entity Types:
{'PERSON', 'TIME', 'PRODUCT', 'NORP', 'GPE', 'LAW', 'ORDINAL', 'MONEY', 'PERCENT', 'LANGUAGE', 'LOC', 'ORG', 'FAC', 'WORK_OF_ART', 'CARDINAL', 'QUANTITY', 'EVENT', 'DATE'}


QUESTION 3:How many entities of each type are mentioned in the reviews?

In [33]:
# Initialize a Counter to count entity type occurrences
entity_type_counter = Counter()

# Process reviews file and count named entity types
with open(reviews_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i >= 1000:
            break
        review_text = line.strip()  # Assuming each line contains a review text
        doc = nlp(review_text)
        # Increment counter for each entity type in the document
        for ent in doc.ents:
            entity_type_counter[ent.label_] += 1

# Print the count of entities for each type
print("Entity Type Counts:")
for entity_type, count in entity_type_counter.items():
    print(f"{entity_type}: {count}")


Entity Type Counts:
ORG: 1282
PERSON: 293
CARDINAL: 767
TIME: 267
ORDINAL: 98
FAC: 11
DATE: 861
PERCENT: 17
GPE: 73
MONEY: 50
LOC: 23
WORK_OF_ART: 25
NORP: 11
EVENT: 5
PRODUCT: 20
LAW: 1
QUANTITY: 3
LANGUAGE: 1
